In [1]:
# Author: Alexander Maksiaev
# Purpose: Generate questions for the Qualtrics surveys in which humans rate valence of words.

In [2]:
# Housekeeping

import os
import pandas as pd
import csv 
import docx
from english_words import get_english_words_set
import enchant
import inflect

web2lowerset = get_english_words_set(['web2'], lower=True)

os.getcwd()

textbook_dir = r"C:\Users\maksi\Documents\UVA\Research\DMP\Textbook_Dump"

textbooks = os.listdir(textbook_dir)

In [3]:
os.chdir(textbook_dir)

# Function to get full text
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return fullText

# Dictionary filled with text for all the books, minus the titles
title_text = {}
for book in textbooks:
    total_text = getText(book)
    text_without_title = total_text[4:]
    for piece in text_without_title:
        if piece == '':
            text_without_title.remove(piece) # Does not get rid of all whitespace, but ah well.
#     text_without_title.remove('')
    title_text[book] = text_without_title

In [4]:
# Remove stop words from each book

stopwords_dir = r"C:\Users\maksi\Documents\UVA\Research\DMP\Stop_Words" 

os.chdir(stopwords_dir)

f = open("stop_words_english_original.txt", "r", encoding="utf-8")
stopwords = []
for text in f:
    text = text.replace('\n', '')
    stopwords.append(text)


punc = '''!()[]{};:'-"\,<>./?@#$%^&*_~''' # Must include "-" in words... or not?

# Function to clean up text and remove stopwords
def clean(book):
    text_list = title_text[book]
    new_text_list = []
    
    # Clean up text
    for text in text_list:
        text = text.strip()
        text = text.lower()
        text = text.split(' ')
        new_text_list.append(text)


    newer_text_list = []
    
    # Remove punctuation
    for sentence in new_text_list:
        for word in sentence:
            for char in punc:
                if char in word:
                    word = word.replace(char, '')
            newer_text_list.append(word)
                
    # Remove stop words
    newest_text_list = []
    for words in newer_text_list:
        if words not in stopwords:
            newest_text_list.append(words)

    # Remove blanks
    for w in newest_text_list:
        if len(w) == 0:
            newest_text_list.remove(w)
            
    return newest_text_list

In [5]:
# Update all the books with their clean, stopword-less counterparts

clean_texts = {}
for book in title_text:
    newest_text_list = clean(book)
    clean_texts[book] = newest_text_list
    
print(clean_texts["Abn_Nolen-Hoeksema_05_Autism_v2.docx"])

['29', 'people', 'biological', 'approach', 'traditionally', 'accepted', 'continuum', 'model', 'abnormality', 'viewing', 'psychological', 'disorders', 'schizophrenia', 'absent', 'view', 'medical', 'physical', 'disorders', 'cancer', 'biological', 'approaches', 'begun', 'view', 'disorders', 'spectrum', 'cannon', 'keller', '2006', 'disorders', 'symptoms', 'varying', 'intensity', 'autism', 'disorder', 'characterized', 'problems', 'communication', 'social', 'skills', 'activities', 'interests', 'suggests', 'spectrum', 'disorders', 'symptoms', 'vary', 'severity', 'disorder', 'severe', 'spectrum', 'aspergers', 'syndrome', 'characterized', 'problems', 'social', 'skills', 'activities', 'autism', 'severe', 'communication', 'difficulties', 'researchers', 'speak', 'autism', 'spectrum', 'disorders', 'sigman', 'spence', 'wang', '2006', 'disorders', 'spectrum', 'qualitatively', 'normal', 'functioning', '297', 'dr', 'temple', 'grandin', 'professor', 'animal', 'sciences', 'colorado', 'university', 'desig

In [6]:
d = enchant.Dict("en_US")

p = inflect.engine()

In [7]:
total_unique_words = set()
for book in clean_texts:
    text_list = clean_texts[book]
    for i in text_list:
        if len(i) > 0 and d.check(i) and (i in web2lowerset or d.check(p.plural(i))):
            total_unique_words.add(i)

In [8]:
all_words = list(total_unique_words)
print(all_words)

['deduce', 'deb', 'goal', 'antioxidants', 'calculate', 'condemn', 'gun', 'fearfully', 'intermittent', 'port', 'organisms', 'affection', 'tumors', 'ideological', 'nowadays', 'immense', 'albeit', 'recurring', 'staff', 'bushel', 'strangely', 'growing', 'preventative', 'subject', 'nonsensical', 'proportion', 'forehead', 'thrown', 'fiction', 'blades', 'dominated', 'ally', 'vital', 'pragmatics', 'suggesting', 'reminders', 'unnecessary', 'periodically', 'man', 'area', 'qualifier', 'strengths', 'rubella', 'mitochondria', 'quarters', 'introductory', 'suitable', 'informative', 'unit', 'remarks', 'politicians', 'meat', 'facilitative', 'pranks', 'combine', 'excited', 'arts', 'reside', 'disruption', 'carrying', 'scope', 'fraud', 'rotate', 'dumpers', 'newspaper', 'formula', 'husbands', 'parrot', 'intentioned', 'matrices', 'wrote', 'discreet', 'prohibits', 'groups', 'minuscule', 'graduated', 'phonological', 'forces', 'abuse', 'closely', 'businesses', 'lithium', 'provide', 'fuss', 'fruits', 'publishes

In [9]:
surveys_dir = r"C:\Users\maksi\Documents\UVA\Research\DMP\Qualtrics_Surveys" 

os.chdir(surveys_dir)

In [10]:
# Build the survey

# for i in range(48):
#     string_i = str(i)
#     name_of_file = 'survey_' + string_i + '.txt'
#     file = open(name_of_file,'w')
    
#     file.write('[[AdvancedFormat]]'+'\n\n' + "[[Question:MC:SingleAnswer:Horizontal]]" + '\n' + "On the following pages, please rate each word's valence (that is, how positive or negative that word seems according to its definition) from a scale of -4 to +4, where -4 is extremely negative, 0 is completely neutral, and +4 is extremely positive.<br><br>Please choose the option you think <i>most other people</i> would choose.<br><br>You may come across words that seem like duplicates. For example, you may encounter a word that is simply the plural version of a word you saw before. Please answer these words as you would normally.<br><br>If you do not know the definition of a word, please choose 0." + '\n\n' + "[[Block:MC Block]]" + '\n\n')
    
    
#     test_words = ["stutter", "depriving", "charming", "backed", "resolvable", "proud", "shy", "repulse", "ungrateful", "slavery"]
    
#     # Test words
#     for item in test_words:
#         file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+item+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
#     file.write("[[Block:MC Block]]" + '\n\n')
    
#     all_words_first = i*198
    
#     if all_words_first != 0:
#         all_words_first += 1
    
#     all_words_last = (i+1)*198
    
#     one = int(((i+1)*198)/33)
    
#     two = int(((i+1)*198)/33 + 33)
    
#     three = int(((i+1)*198)/33 + 33*2)
    
#     four = int(((i+1)*198)/33 + 33*3)
    
#     five = int(((i+1)*198)/33 + 33*4)
     
#     six = int(((i+1)*198)/33 + 33*5)
        
#     file.write("[[Question:MC:SingleAnswer:Horizontal]]\n" + "Once again, please rate each word's valence (that is, how positive or negative that word seems according to its definition) from a scale of -4 to +4, where -4 is extremely negative, 0 is completely neutral, and +4 is extremely positive.<br><br>Please choose the option you think <i>most other people</i> would choose.\n\n")
    
#     # Actual textbook words
#     for item in all_words[all_words_first:one]:
#         file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+item+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
#     file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+"Please choose +2.\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
#     file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+"love"+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
        
#     for item in all_words[one:two]:
#         file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+item+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
       
#     file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+"safe"+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
#     file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+"Please choose -1."+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
    
#     for item in all_words[two:three]:
#         file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+item+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
       
#     file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+"prepared"+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
#     for item in all_words[three:four]:
#         file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+item+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
       
#     file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+"reach"+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
#     file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+"Please choose 0."+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
    
#     for item in all_words[four:five]:
#         file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+item+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
       
#     file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+"cruelness"+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
#     for item in all_words[five:six]:
#         file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+item+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
      
#     file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+"terrorist"+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
#     file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+"Please choose +3."+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
    
#     for item in all_words[six:all_words_last]:
#         file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+item+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
      
        
#     file.close()

In [14]:
file = open('giant_survey.txt','w')
# file.write('[[AdvancedFormat]]'+'\n\n')
# file = open("beginning.txt",'w')
    
file.write('[[AdvancedFormat]]'+'\n\n[[Block]]\n\n' + "[[Question:MC:SingleAnswer:Horizontal]]" + '\n' + "On the following pages, please rate each word's valence (that is, how positive or negative that word is according to its definition) from a scale of extremely negative to extremely positive.<br><br>Please choose the option you think <i>most other people</i> would choose.<br><br>You may come across words that seem like duplicates. For example, you may encounter a word that is simply the plural version of a word you saw before. Please answer these words as you would normally.<br><br>If you do not know the definition of a word, please choose 0." + '\n\n') # + "[[Block:MC Block]]" + '\n\n')
    
    
test_words = ["stutter", "depriving", "charming", "backed", "resolvable", "proud", "shy", "repulse", "ungrateful", "slavery"]
    
# Test words
for item in test_words:
    file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+item+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
file.write("[[PageBreak]]\n\n")
    
file.close()    

In [17]:

    
#     test_words = ["stutter", "depriving", "charming", "backed", "resolvable", "proud", "shy", "repulse", "ungrateful", "slavery"]
    
#     # Test words
#     for item in test_words:
#         file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+item+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
#     file.write("[[Block:MC Block]]" + '\n\n')
for i in range(48):
    
    string_i = str(i)
    name_of_file = 'survey_' + string_i + '.txt'
    file = open(name_of_file,'w')

    file.write('[[AdvancedFormat]]\n\n' + "[[Block]]\n\n"+'\n\n' + "[[Question:MC:SingleAnswer:Horizontal]]" + '\n' + "Once again, please rate each word's valence (that is, how positive or negative that word seems according to its definition) from a scale of -4 to +4, where -4 is extremely negative, 0 is completely neutral, and +4 is extremely positive.<br><br>Please choose the option you think <i>most other people</i> would choose.<br><br>You may come across words that seem like duplicates. For example, you may encounter a word that is simply the plural version of a word you saw before. Please answer these words as you would normally.<br><br>If you do not know the definition of a word, please choose 0." + '\n\n') # + "[[Block:MC Block]]" + '\n\n')
    

    all_words_first = i*198
    
#     if all_words_first != 0:
#         all_words_first += 1
#     print(all_words_first, " ", i)
    
    all_words_last = (i+1)*198
#     print(all_words_last, " ", i)
    
#     one = int(((i+1)*198)-33)
#     print(one)

    one = int(((i+1))*198 - (33*0.5))
    
    two = int(((i+1)*198)-(33))
#     print(two)

    twothree = int(((i+1)*198)-(33*1.5))
    
    three = int(((i+1)*198)-(33*2))
#     print(three)
    threefour = int(((i+1)*198) - (33*2.5))
    
    four = int(((i+1)*198)-(33*3))
#     print(four)

    fourfive = int(((i+1)*198)-(33*3.5))
    
    five = int(((i+1)*198)-(33*4))
#     print(five)

    fivesix = int(((i+1)*198) - (33*4.5))
     
    six = int(((i+1)*198)-(33*5))
    
    sixsix = int(((i+1)*198)-(33*5.5))
#     print(six)
        
#     file.write("[[Question:MC:SingleAnswer:Horizontal]]\n" + "Once again, please rate each word's valence (that is, how positive or negative that word seems according to its definition) from a scale of -4 to +4, where -4 is extremely negative, 0 is completely neutral, and +4 is extremely positive.<br><br>Please choose the option you think <i>most other people</i> would choose.\n\n")
    
    # Actual textbook words
    for item in all_words[all_words_first:sixsix]:
        file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+item+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
    file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+"Please choose the option [+2].\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
    file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+"love"+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
        
    file.write("[[PageBreak]]\n\n")
        
    for item in all_words[sixsix:six]:
        file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+item+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
    file.write("[[PageBreak]]\n\n")
        
    for item in all_words[six:fivesix]:
        file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+item+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
       
    file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+"safe"+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
    file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+"Please choose the option [-1]."+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
    file.write("[[PageBreak]]\n\n")
    
    for item in all_words[fivesix:five]:
        file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+item+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
       
    file.write("[[PageBreak]]\n\n") 
    
    for item in all_words[five:fourfive]:
        file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+item+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
       
    file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+"prepared"+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
    file.write("[[PageBreak]]\n\n") 
    
    for item in all_words[fourfive:four]:
        file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+item+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
       
    file.write("[[PageBreak]]\n\n") 
    
    for item in all_words[four:threefour]:
        file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+item+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
       
    file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+"reach"+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
    file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+"Please choose the option [0]."+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
    file.write("[[PageBreak]]\n\n") 
    
    for item in all_words[threefour:three]:
        file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+item+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
    file.write("[[PageBreak]]\n\n") 
    
    for item in all_words[three:twothree]:
        file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+item+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
       
    file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+"cruelness"+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
    file.write("[[PageBreak]]\n\n") 
    
    for item in all_words[twothree:two]:
        file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+item+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
       
    
    file.write("[[PageBreak]]\n\n") 
    
    for item in all_words[two:one]:
        file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+item+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
       
    file.write("[[PageBreak]]\n\n")
    
    for item in all_words[one:all_words_last]:
        file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+item+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
      
    file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+"terrorist"+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
    file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+"Please choose the option [+3]."+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
    
    file.write("[[PageBreak]]\n\n")
    
#     for item in all_words[one:all_words_last]:
#         file.write("[[Question:MC:SingleAnswer:Horizontal]]\n"+item+"\n" + "[[AdvancedChoices]]\n" + "[[Choice:-4]]\n[-4] Extremely negative\n" + "[[Choice:-3]]\n[-3] Very negative\n" + "[[Choice:-2]]\n[-2] Moderately negative\n" + "[[Choice:-1]]\n[-1] Slightly negative\n" + "[[Choice:0]]\n[0] Neutral (or neither, N/A)\n" + "[[Choice:1]]\n[+1] Slightly positive\n" + "[[Choice:2]]\n[+2] Moderately positive\n" + "[[Choice:3]]\n[+3] Very positive\n" + "[[Choice:4]]\n[+4] Extremely positive\n" + "\n")
      
        
file.close()